<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/ETL_C670.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import warnings 
warnings.simplefilter('ignore')
def get_col(df,name):
    return df.columns[df.columns.str.contains(name)].tolist()

In [2]:
path = '/content/drive/MyDrive/台塑輕油案子/data/c620/raw/c670/Aroma-2 C670_Split Factors Calculation for AI Training_Overall_Data_new.xlsx'

In [3]:
xl = pd.ExcelFile(path)

In [4]:
def clean_data(df):
    df = df.T.reset_index()
    idx = df.iloc[6:,1].values
    df = df.iloc[:,2:]
    prefix = pd.Series(df.iloc[1,:].values).fillna(method='ffill').values
    middle = pd.Series(df.iloc[2,:].values).fillna(method='ffill').values
    middle_2 = pd.Series(df.iloc[3,:].values).fillna(method='ffill').values
    suffix = pd.Series(df.iloc[4,:].values).fillna(method='ffill').values
    col = [p.strip() + '_' + m1.strip() + '_' + m2.strip() + '_' + s.strip() for p,m1,m2,s in zip(prefix,middle,middle_2,suffix)]
    df = df.iloc[6:,:]
    df.index = idx
    df.columns = col
    return df.astype('float')

In [5]:
train = pd.DataFrame()
for i in tqdm(xl.sheet_names):
    rows = clean_data(xl.parse(i))
    if len(train) == 0:
      train = rows
    else:
      train = train.append(rows)

In [6]:
train.head()

,Combined Feed to Toluene Column C670 @ Tray 43_Flow Rate_Volume Flow Rate_m3/hr,Combined Feed to Toluene Column C670 @ Tray 43_Flow Rate_Mass Flow Rate_ton/hr,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Hydrogen_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Methane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Ethane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Propane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Butane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Pentane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Hexane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Benzene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Cyclohexane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Heptane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Water_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Methylcyclohexane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Toluene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Octane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Propylcyclopentane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Ethylcyclohexane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Ethylbenzene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_p-Xylene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_m-Xylene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_o-Xylene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Nonane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_i-Propylbenzene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Propylcyclohexane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Propylbenzene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1-Methyl-3-ethylbenzene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1-Methyl-4-ethylbenzene_wt%,"Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1,3,5-Trimethylbenzene_wt%",Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1-Methyl-2-ethylbenzene_wt%,"Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1,2,4-Trimethylbenzene_wt%",Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_tert-Butylcyclohexane_wt%,"Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1,2,3-Trimethylbenzene_wt%",Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Indane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1-Methyl-4-n-propylbenzene_wt%,"Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1,2-Diethylbenzene_wt%",Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_5-Ethyl-m-xylene_wt%,"Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1,4-Diethylbenzene_wt%","Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1,2,3,5-Tetramethylbenzene_wt%",Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Pentylbenzene_wt%,...,Toluene Column C670 Feed Blending Factors Calculation_Lower Feed Blending Factor @ Tray 54 for Individual Component to Toluene Column C670_Methane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Lower Feed Blending Factor @ Tray 54 for Individual Component to Toluene Column C670_Ethane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Lower Feed Blending Factor @ Tray 54 for Individual Component to Toluene Column C670_Propane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Lo

In [7]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
# Missing values statistics
missing_values = missing_values_table(train)
missing_values.head(20)

Your selected dataframe has 429 columns.
There are 7 columns that have missing values.


,Missing Values,% of Total Values
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Sum_wt%,15552,100.0
Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Sum_wt%,15552,100.0
Toluene Column C670 Split Factors Calculation_Split Factor for Individual Component to Toluene Column C670 Distillate_Dist Split Factor_wt%,15552,100.0
Toluene Column C670 Split Factors Calculation_Split Factor for Individual Component to Toluene Column C670 Bottoms_Btms Split Factor_Fraction,15552,100.0
Toluene Column C670 Split Factors Calculation_Split Factor for Individual Component to Toluene Column C670 Bottoms_Oxygen_Fraction,15552,100.0
Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Upper Feed Blending Factor_Fraction,15552,100.0
Toluene Column C670 Feed Blending Factors Calculation_Lower Feed Blending Factor @ Tray 54 for Individual Component to Toluene Column C670_Lower Feed Blending Factor_Fraction,15552,100.0


In [8]:
print(train.shape)
train = train.dropna(axis=1)
train.shape

(15552, 429)


(15552, 422)

#檢查mass balance

In [9]:
fr = get_col(train,'Mass Flow Rate')[2:]
check_df = train[fr]
check_df.head()
assert np.allclose(check_df.iloc[:,:2].sum(axis=1).values,check_df.iloc[:,2:].sum(axis=1).values) 
check_df.to_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/{}_mbcheck.csv'.format('c670'))
list(fr)

['Toluene Column C670 Operation_Feed Properties \n@ Tray 43_Mass Flow Rate_ton/hr',
 'Toluene Column C670 Operation_Feed Properties \n@ Tray 54_Mass Flow Rate_ton/hr',
 'Toluene Column C670 Operation_Distillate Production Rate and Composition_Mass Flow Rate_ton/hr',
 'Toluene Column C670 Operation_Bottoms Production Rate and Composition_Mass Flow Rate_ton/hr']

In [10]:
xTray43 = get_col(train,'Tray 43_Detailed')[:-1]
xTray54 = get_col(train,'Tray 54_Detailed')[:-1]

combined = get_col(train, 'Combined Feed to Toluene Column C670 : Detailed weight % Composition')[1:-1]

upper_bf = get_col(train,'Upper Feed Blending Factor @ Tray 43 for Individual Component')
lower_bf = get_col(train,'Lower Feed Blending Factor @ Tray 54 for Individual Component')

yRefluxRate = get_col(train, 'Reflux Rate')
yControl = get_col(train, '(Control)')
yHeatDuty = get_col(train, 'Heat Duty_Mkcal')

distillate_x = get_col(train,'Operation_Distillate')[2:-1]
bottoms_x = get_col(train,'Operation_Bottoms')[2:-1]

distillate_sf = get_col(train,'C670 Distillate')
bottoms_sf = get_col(train,'C670 Bottoms')

for i in [xTray43,xTray54,combined,upper_bf,lower_bf,yRefluxRate,yControl,yHeatDuty,distillate_x,bottoms_x,distillate_sf,bottoms_sf]:
  print(len(i))

41
41
41
41
41
1
2
2
41
41
41
41


In [11]:
import numpy as np
def sp2wt(x,s):
  a = 100*x*s
  b = np.diag(x@s.T).reshape(-1,1)
  b = np.clip(b,1e-8,np.inf)
  return a/b

In [12]:
for sf,wt in tqdm(zip([distillate_sf,bottoms_sf],[distillate_x,bottoms_x])):
  assert np.allclose(sp2wt(train[combined].values,train[sf].values),train[wt].values)

In [13]:
col_names = {}
col_names['xTray43'] = xTray43
col_names['xTray54'] = xTray54
col_names['combined'] = combined
col_names['upper_bf'] = upper_bf
col_names['lower_bf'] = lower_bf

col_names['yRefluxRate'] = yRefluxRate
col_names['yControl'] = yControl
col_names['yHeatDuty'] = yHeatDuty

col_names['distillate_x'] = distillate_x
col_names['bottoms_x'] = bottoms_x

col_names['distillate_sf'] = distillate_sf
col_names['bottoms_sf'] = bottoms_sf

In [14]:
MFR = get_col(train, 'Mass Flow Rate')[-2:]
VFR = get_col(train, 'Volume Flow Rate')[-2:]
vMFR = train[MFR].values
vVFR = train[VFR].values
n = [ i.split('_')[1] for i in MFR]
col_names['density'] = ['Density_{}'.format(i) for i in n]
density = vMFR/vVFR
density = pd.DataFrame(density,columns=col_names['density'])
density.head()

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition
0,0.870898,0.872382
1,0.870897,0.872382
2,0.870897,0.872382
3,0.870898,0.872382
4,0.870897,0.872382


In [15]:
col_names.keys()

dict_keys(['xTray43', 'xTray54', 'combined', 'upper_bf', 'lower_bf', 'yRefluxRate', 'yControl', 'yHeatDuty', 'distillate_x', 'bottoms_x', 'distillate_sf', 'bottoms_sf', 'density'])

In [16]:
train = train.join(density)
print(train.shape)

(15552, 424)


In [17]:
for k,v in tqdm(col_names.items()):
  assert len(train[v]) != 0

In [18]:
train.head()

,Combined Feed to Toluene Column C670 @ Tray 43_Flow Rate_Volume Flow Rate_m3/hr,Combined Feed to Toluene Column C670 @ Tray 43_Flow Rate_Mass Flow Rate_ton/hr,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Hydrogen_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Methane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Ethane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Propane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Butane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Pentane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Hexane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Benzene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Cyclohexane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Heptane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Water_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Methylcyclohexane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Toluene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Octane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Propylcyclopentane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Ethylcyclohexane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Ethylbenzene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_p-Xylene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_m-Xylene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_o-Xylene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Nonane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_i-Propylbenzene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Propylcyclohexane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Propylbenzene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1-Methyl-3-ethylbenzene_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1-Methyl-4-ethylbenzene_wt%,"Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1,3,5-Trimethylbenzene_wt%",Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1-Methyl-2-ethylbenzene_wt%,"Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1,2,4-Trimethylbenzene_wt%",Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_tert-Butylcyclohexane_wt%,"Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1,2,3-Trimethylbenzene_wt%",Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_Indane_wt%,Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1-Methyl-4-n-propylbenzene_wt%,"Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1,2-Diethylbenzene_wt%",Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_5-Ethyl-m-xylene_wt%,"Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1,4-Diethylbenzene_wt%","Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_1,2,3,5-Tetramethylbenzene_wt%",Combined Feed to Toluene Column C670 @ Tray 43_Detailed Composition_n-Pentylbenzene_wt%,...,Toluene Column C670 Feed Blending Factors Calculation_Lower Feed Blending Factor @ Tray 54 for Individual Component to Toluene Column C670_Propane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Lower Feed Blending Factor @ Tray 54 for Individual Component to Toluene Column C670_n-Butane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Lower Feed Blending Factor @ Tray 54 for Individual Component to Toluene Column C670_n-Pentane_Fraction,Toluene Column C670 Feed Blending Factors Calculatio

In [19]:
import joblib
train.to_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/{}_train.csv'.format('c670'))
joblib.dump(col_names,'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/{}_col_names.pkl'.format('c670'))
print('ok')

ok
